In [2]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


### Загрузим данные с логов

In [5]:
from logParser import ParseLogsFromFolder

main_data, values_data, order_data = ParseLogsFromFolder('Logs/', 0, 10, only_order=False)

main = pd.DataFrame(main_data)
del(main_data)
# del(values_data)
print('Shape: ' + str(main.shape))
main.head()


  8%|▊         | 10/127 [00:12<02:31,  1.30s/it]

Shape: (289269, 3)


,User_Agent,id,ip
0,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,1485900038,95.181.252.91
1,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,1485900047,40.77.167.74
2,Mozilla/5.0 (Linux; Android 4.4.2; Zera S Buil...,1485900079,31.148.3.19
3,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,1485900103,188.162.183.66
4,Mozilla/4.0 (compatible; MSIE 8.0; Windows NT ...,1485900105,37.144.52.103


  8%|▊         | 10/127 [00:28<05:31,  2.84s/it]

### Выберем нужные столбцы из матрицы со значениями

In [13]:
good_value_keys = {'Accept', 'Accept-Encoding', 'Accept-Charset'}
good_values_data = []
for record in tqdm(values_data, mininterval=1.2):
    good_values_data.append(
        {key: value for key, value in record.items() if key in good_value_keys})
good_values_data = pd.DataFrame(good_values_data)
# good_values_data.fillna('', inplace=True)
del(values_data)
good_values_data.head()


100%|██████████| 289269/289269 [00:01<00:00, 267161.32it/s]


,Accept,Accept-Charset,Accept-Encoding
0,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate, sdch"
1,*/*,,"gzip, deflate"
2,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate, sdch"
3,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate"
4,*/*,,"gzip, deflate"


### На их основе составим дамми признаки

In [17]:
good_values_data

,Accept,Accept-Charset,Accept-Encoding
0,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate, sdch"
1,*/*,,"gzip, deflate"
2,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate, sdch"
3,"text/html,application/xhtml+xml,application/xm...",,"gzip, deflate"
4,*/*,,"gzip, deflate"
...,...,...,...
289264,"text/html,text/plain,text/xml,text/*,applicati...",,gzip
289265,"text/html,text/plain,text/xml,text/*,applicati...",,gzip
289266,"text/html,text/plain,text/xml,text/*,applicati...",,gzip
289267,*/*,,"gzip, deflate"


In [16]:
main_top_100 = main[main.User_Agent.isin(main.User_Agent.value_counts()[:100].index)]
#del(main)

top_ua = main.User_Agent.value_counts()[:100].index.tolist()

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
sparse_orders = orders_vectorizer.fit_transform(order_data).astype(np.int8)
del(order_data)